In [1]:
import os
import json
import pprint
import copy

os.chdir('../src/meshflow/')
os.getcwd()

'/home/kasra.keshavarz1/github-repos/meshflow/src/meshflow'

In [2]:
# # necessary functions
def deep_merge(d1, d2):
    """
    Recursively merge d2 into d1.
    """
    for key, value in d2.items():
        if key in d1 and isinstance(d1[key], dict) and isinstance(value, dict):
            deep_merge(d1[key], value)  # Recursive merge for nested dictionaries
        else:
            d1[key] = value  # Overwrite or add new key-value pair
    return d1

In [3]:
info_block = {
    "vars": {
        "case_name": "MESH for Bow River at Calgary",
        "author": "Kasra Keshavarz",
        "location": "University of Calgary",
    },
    "comments": {
        "case_name": "01 TITLE",
        "author": "02 NAME",
        "location": "03 PLACE"
    },
    "formats": {}, # none means no change
    "columns": {
        "case_name": 3,
        "author": 3,
        "location": 3,
    }, # none means column 2
}

case_block = {
    "vars": {
        "case": {
            "centroid_lat": 45.2,
            "centroid_lon": -114.2,
            "reference_height_wndspd": 40.0,
            "reference_height_spechum_airtemp": 40.0,
            "reference_height_surface_roughness": 50.0,
            "ground_cover_flag": -1.0, # meaning land cover
            "ILW": 1, # MESH,CLASS>3.4 defaults to 1
            "NL": 30, # total number of subbasins
            "NM": 5, # total number of GRUs - 1
        },
    },
    "comments": {
        "case": '04 DEGLAT/DEGLON/ZRFM/ZRFH/ZBLD/GC/ILW/NL/NM'
    },
    "formats": {
        "case": {
            "centroid_lat": 2,
            "centroid_lon": 2,
            "reference_height_wndspd": 1,
            "reference_height_spechum_airtemp": 1,
            "reference_height_surface_roughness": 1,
            "ground_cover_flag": 1,
            "ILW": 0,
            "NL": 0,
            "NM": 0,
        },
    },
    "columns": {
        "case": {
            "centroid_lat": 4,
            "centroid_lon": 12,
            "reference_height_wndspd": 25,
            "reference_height_spechum_airtemp": 35,
            "reference_height_surface_roughness": 45,
            "ground_cover_flag": 52,
            "ILW": 60,
            "NL": 65,
            "NM": 70,
        },
    },
}

gru_block = {
    "vars": [
        {
            "veg": {
                "class": "needleleaf",
            },
            "hyd": {
                "line13": {
                    "mid": "1 Needleleaf Forest (sub-polar)",
                },
            },
        },
        {
            "veg": {
                "class": "broadleaf",
            },
            "hyd": {
                "line13": {
                    "mid": "2 Broadleaf Forest (Taiga)",
                },
            },
        },
        {
            "veg": {
                "class": "crops",
            },
            "hyd": {
                "line13": {
                    "mid": "3 Cropland",
                },
            },
        },
        {
            "veg": {
                "class": "grass",
            },
            "hyd": {
                "line13": {
                    "mid": "4 Grassland-moss",
                },
            },
        },
        {
            "veg": {
                "class": "barrenland",
            },
            "hyd": {
                "line13": {
                    "mid": "5 barrenland",
                },
            },
        },
    ],
}

In [4]:
# load the default values for each GRU
with open('./templates/default_parameters.json', 'r') as file:
    data = json.load(file)

# populate new dictionary for blocks
populating_list = []

# deep update GRU blocks
for idx, block in enumerate(gru_block['vars']):
    new_data = copy.deepcopy(data)

    # deep merge
    it = deep_merge(new_data['class_defaults'], block)
    
    # update the block dictionary
    populating_list.append(it)

gru_block.update({'vars': populating_list})

# add formats, columns, and comments
new_keys = ['formats', 'comments', 'columns']
for key in new_keys:
    gru_block[key] = data[key]

In [5]:
pprint.pprint(gru_block)

{'columns': {'hyd': {'line12': 78, 'line13': 78},
             'prog': {'line17': 78, 'line18': 78, 'line19': 78},
             'soil': {'line14': 78, 'line15': 78, 'line16': 78},
             'veg': {'line10': 78,
                     'line11': 78,
                     'line5': 78,
                     'line6': 78,
                     'line7': 78,
                     'line8': 78,
                     'line9': 78}},
 'comments': {'line10': '10 4xVPDA/4xVPDB',
              'line11': '11 4xPSGA/4xPSGB',
              'line12': '12 DRN/SDEP/FARE/DD',
              'line13': '13 XSLP/XDRAINH/MANN/KSAT/MID',
              'line14': '14 3xSAND (or more)',
              'line15': '15 3xCLAY (or more)',
              'line16': '16 3xORGM (or more)',
              'line17': '17 3xTBAR (or more)/TCAN/TSNO/TPND',
              'line18': '18 3xTHLQ (or more)/3xTHIC (or more)/ZPND',
              'line19': '19 RCAN/SCAN/SNO/ALBS/RHOS/GRO',
              'line5': '05 5xFCAN/4xLAMX',
             

In [8]:
from jinja2 import Environment, FileSystemLoader

def raise_helper(msg):
    raise Exception(msg)

environment = Environment(
    loader=FileSystemLoader("templates/"),
    trim_blocks=True,
    lstrip_blocks=True,
    line_comment_prefix='##',
)
environment.globals['raise'] = raise_helper

template = environment.get_template("MESH_parameters_CLASS.ini.jinja")

# create content
content = template.render(
    info_block=info_block,
    case_block=case_block,
    gru_block=gru_block,
    variables="vars",
    comments="comments",
    formats="formats",
    columns="columns",
)

print(content)

  MESH for Bow River at Calgary                                              01 TITLE
  Kasra Keshavarz                                                            02 NAME
  University of Calgary                                                      03 PLACE
   45.20   -114.20      40.0      40.0      50.0   -1.0    1    30   5       04 DEGLAT/DEGLON/ZRFM/ZRFH/ZBLD/GC/ILW/NL/NM
   1.000   0.000   0.000   0.000   0.000   1.450   0.000   0.000   0.000     05 5xFCAN/4xLAMX
  -1.300   0.000   0.000   0.000   0.000   1.200   0.000   0.000   0.000     06 5xLNZ0/4xLAMN
   0.045   0.000   0.000   0.000   0.000   4.500   0.000   0.000   0.000     07 5xALVC/4xCMAS
   0.160   0.000   0.000   0.000   0.000   1.090   0.000   0.000   0.000     08 5xALIC/4xROOT
 145.000   0.000   0.000   0.000          36.000   0.000   0.000   0.000     09 4xRSMN/4xQA50
   0.800   0.000   0.000   0.000           1.050   0.000   0.000   0.000     10 4xVPDA/4xVPDB
 100.000   0.000   0.000   0.000           5.000   0.000 